In [53]:
import csv
import json
import pandas as pd
import numpy as np

In [55]:
# Define filepaths for import
countriesFilePath = r'C:\Users\billy\Desktop\Sep152021\group_project\Group_Resources\countries.csv'
cleanedFilePath = r'C:\Users\billy\Desktop\Sep152021\group_project\ETL\model_dataset_cleaned.csv'
initjsonFilePath = r'C:\Users\billy\Desktop\Sep152021\group_project\Dashboard\static\js\countryBoundries.json'
jsonOutputFilePath = r'C:\Users\billy\Desktop\Sep152021\group_project\Dashboard\static\js\countryDataBoundries.json'

In [56]:
# Load GeoJson data for update
file = open(initjsonFilePath)
countryBoundries = json.load(file)

In [57]:
# Load cleaned dataset to pull data for json file update
df_cleaned = pd.read_csv(cleanedFilePath)
df_cleaned.head()

,id,name,nationality,country,sex,dob,height,weight,sport,gold,silver,bronze,population,gdp_per_capita,dob_converted,age,total_medals,placed
0,736041664,A Jesus Garcia,ESP,Spain,male,10/17/1969,1.72,64,athletics,0,0,0,46418269,25832,10/17/2069,46,0,0
1,532037425,A Lam Shin,KOR,"Korea, South",female,9/23/1986,1.68,56,fencing,0,0,0,50617045,27222,9/23/1986,29,0,0
2,435962603,Aaron Brown,CAN,Canada,male,5/27/1992,1.98,79,athletics,0,0,1,35851774,43249,5/27/1992,24,1,1
3,521041435,Aaron Cook,MDA,Moldova,male,1/2/1991,1.83,80,taekwondo,0,0,0,3554150,1848,1/2/1991,25,0,0
4,33922579,Aaron Gate,NZL,New Zealand,male,11/26/1990,1.81,71,cycling,0,0,0,4595700,37808,11/26/1990,25,0,0


In [58]:
def defineFeatures(groupFunction, dfInput, grouping, calcColumn):
    if groupFunction == 'count':
        df_defineFeatures = (dfInput.groupby(grouping)
                            .count()
                            .reset_index()[[grouping, calcColumn]]
                            .set_index(grouping)
                    )
    else:
        df_defineFeatures = (dfInput.groupby(grouping)
                                    .agg(groupFunction)
                                    .reset_index()[[grouping, calcColumn]]
                                    .set_index(grouping)
                            )

    dict_defineFeatures = df_defineFeatures.to_dict(orient='dict')[calcColumn]    
    
    return [df_defineFeatures, dict_defineFeatures]

In [59]:
defineFeatures('sum', df_cleaned, 'nationality', 'total_medals')[1]

{'AFG': 0,
 'ALB': 0,
 'ALG': 2,
 'ANG': 0,
 'ANT': 0,
 'ARG': 22,
 'ARM': 4,
 'AUS': 82,
 'AUT': 2,
 'AZE': 16,
 'BAH': 3,
 'BAN': 0,
 'BAR': 0,
 'BDI': 1,
 'BEL': 21,
 'BEN': 0,
 'BHU': 0,
 'BIH': 0,
 'BIZ': 0,
 'BLR': 12,
 'BOL': 0,
 'BOT': 0,
 'BRA': 50,
 'BRN': 2,
 'BRU': 0,
 'BUL': 7,
 'BUR': 0,
 'CAF': 0,
 'CAM': 0,
 'CAN': 69,
 'CGO': 0,
 'CHA': 0,
 'CHI': 0,
 'CHN': 109,
 'CIV': 1,
 'CMR': 0,
 'COD': 0,
 'COL': 6,
 'COM': 0,
 'CPV': 0,
 'CRC': 0,
 'CRO': 23,
 'CYP': 0,
 'CZE': 15,
 'DEN': 41,
 'DJI': 0,
 'DMA': 0,
 'DOM': 1,
 'ECU': 0,
 'EGY': 3,
 'ESA': 0,
 'ESP': 45,
 'EST': 4,
 'ETH': 8,
 'FIJ': 13,
 'FIN': 0,
 'FRA': 89,
 'FSM': 0,
 'GAB': 0,
 'GBR': 141,
 'GBS': 0,
 'GEO': 7,
 'GEQ': 0,
 'GER': 159,
 'GHA': 0,
 'GRE': 7,
 'GRN': 0,
 'GUA': 0,
 'GUI': 0,
 'GUY': 0,
 'HAI': 0,
 'HKG': 0,
 'HON': 0,
 'HUN': 22,
 'INA': 4,
 'IND': 2,
 'IRL': 3,
 'IRQ': 0,
 'ISL': 0,
 'ISR': 2,
 'ITA': 72,
 'JAM': 28,
 'JOR': 1,
 'JPN': 65,
 'KAZ': 12,
 'KEN': 12,
 'KGZ': 0,
 'KOR': 26,
 'KSA'

In [60]:
(df_cleaned.groupby('nationality')
    .sum('total_medals')
    .reset_index()[['nationality', 'total_medals']]
    .sort_values('total_medals', ascending=False)
)

,nationality,total_medals
157,USA,258
63,GER,159
59,GBR,141
33,CHN,109
129,RUS,107
...,...,...
66,GRN,0
64,GHA,0
62,GEQ,0
60,GBS,0


In [61]:
# Output list with dataframe and dictionary for required country data
out_totalMedals = defineFeatures('sum', df_cleaned, 'nationality', 'total_medals')
out_countAthlete = defineFeatures('count', df_cleaned, 'nationality', 'name')
out_meanHeight = defineFeatures('mean', df_cleaned, 'nationality', 'height')
out_meanWeight = defineFeatures('mean', df_cleaned, 'nationality', 'weight')
out_meanAge = defineFeatures('mean', df_cleaned, 'nationality', 'age')
out_population = defineFeatures('mean', df_cleaned, 'nationality', 'population')
out_gdp = defineFeatures('mean', df_cleaned, 'nationality', 'gdp_per_capita')

In [62]:
# Parse dictionaries to update JSON file with athlete and country data
dict_totalMedals = out_totalMedals[1]
dict_countAthlete = out_countAthlete[1]
dict_meanHeight = out_meanHeight[1]
dict_meanWeight = out_meanWeight[1]
dict_meanAge = out_meanAge[1]
dict_population = out_population[1]
dict_gdp = out_gdp[1]

In [63]:
# Calculate per capita (millions) medal counts
df_medalsPerCapitaMill = out_totalMedals[0].merge(out_population[0],
                                                  how='left',
                                                  on='nationality',
                                                  validate='many_to_one')
df_medalsPerCapitaMill['medalsPerCapita'] = (1000000 
                                             * (df_medalsPerCapitaMill['total_medals']
                                             /df_medalsPerCapitaMill['population']
                                                )
                                             ).round(4)
dict_medalsPerCapitaMill = df_medalsPerCapitaMill.to_dict(orient='dict')['medalsPerCapita']                                 

In [64]:
# Calculate per gdp medal counts
df_medalsPerGdpThous = out_totalMedals[0].merge(out_gdp[0],
                                           how='left',
                                           on='nationality',
                                           validate='many_to_one')
df_medalsPerGdpThous['medalsPerGdp'] = (1000
                                             * (df_medalsPerGdpThous['total_medals']
                                             /df_medalsPerGdpThous['gdp_per_capita']
                                                )
                                             ).round(4)
dict_medalsPerGdpThous = df_medalsPerGdpThous.to_dict(orient='dict')['medalsPerGdp']                                                   

In [65]:
# Navigate to features in JSON file
dict_features = countryBoundries['features']

In [66]:
# Append JSON dictionary with athlete and country data
for item in dict_features:
    iso = item['properties']['ISO_A3']
    item['properties']['population'] = dict_population.get(iso)
    item['properties']['gdp_per_capita'] = dict_gdp.get(iso)    
    item['properties']['sumMedals'] = dict_totalMedals.get(iso)
    item['properties']['countAlthetes'] = dict_countAthlete.get(iso)
    item['properties']['meanAthHeight'] = dict_meanHeight.get(iso)
    item['properties']['meanAthWeight'] = dict_meanWeight.get(iso)
    item['properties']['meanAthAge'] = dict_meanAge.get(iso)
    item['properties']['MedalsPerCapitaMill'] = dict_medalsPerCapitaMill.get(iso)
    item['properties']['MedalsPerGdpThous'] = dict_medalsPerGdpThous.get(iso)

In [68]:
# Output JSON file
with open(jsonOutputFilePath, 'w', encoding='utf-8') as jsonFile:
    jsonFile.write(json.dumps(dict_features, indent=4))